# Monkey delayed-reaching task

In this notebook, we will visualize data collected from the motor cortex (M1) of a monkey during a delayed reaching task. This data has been preprocessed such that it is aligned to (-100, 450) ms as the monkey reaches toward a target location. 


#### Reference:
Churchland, Mark; Kaufman, Matthew (2022) MC_Maze_Large: macaque primary motor and dorsal premotor cortex spiking activity during delayed reaching (Version 0.220113.0407) [Data set]. DANDI archive. https://doi.org/10.48324/dandi.000138/0.220113.0407

In [ ]:
# imports
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#%matplotlib inline  # uncomment these if jupyter notebook (not lab)
#%matplotlib notebook

import torch
import scipy
import h5py
import pickle
from einops import rearrange

# local imports
import sys
sys.path.append("..")
from code_pack.plotting import raster_to_events

In [ ]:
# loading train data
trial_info_save_path = 'data/info_per_trial_{}.pkl'
spikes_per_trial_save_path = 'data/spikes_per_trial.h5'
rates_per_trial_save_path = 'data/rates_per_trial_{}.npy'
velocity_per_trial_save_path = 'data/velocity_per_trial_{}.npy'
position_per_trial_save_path = 'data/position_per_trial_{}.npy'

spikes_per_trial = h5py.File(spikes_per_trial_save_path, 'r')
spike_data_train = np.array(spikes_per_trial['Y'], np.float32)
# print(spike_data_train.shape)

position_per_trial_train = np.load(position_per_trial_save_path.format("train"))
velocity_per_trial_train = np.load(velocity_per_trial_save_path.format("train"))
rates_per_trial_train = np.load(rates_per_trial_save_path.format("train"))

trial_info_train = []
with (open(trial_info_save_path.format("train"), "rb")) as openfile:
    while True:
        try:
            trial_info_train.append(pickle.load(openfile))
        except EOFError:
            break
trial_info_train = trial_info_train[0]

In [ ]:
# visualizing the raster of the first trial
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
events = raster_to_events(spike_data_train[0,:,:])
_ = ax.eventplot(events, linewidths=0.5, color='blue')

plt.xlabel('Time')
plt.ylabel('Neuron index')

In [ ]:
# Visualize the reaching motion of the monkey for the first 20 trials

fig, ax = plt.subplots(1, 1, figsize=(5, 5))

for i in range(20):
    ax.plot(position_per_trial_train[i,:,0], position_per_trial_train[i,:,1])
    
plt.xlabel('X position')
plt.ylabel('Y position')

plt.grid()

In [ ]:
# adding velocity information as a color on hand motion
mod_vel0 = np.sqrt(velocity_per_trial_train[0,:,0]**2 + velocity_per_trial_train[0,:,1]**2)
mod_vel0 /= np.max(mod_vel0)

cm = plt.get_cmap('plasma')
colors = cm(mod_vel0)
norm = matplotlib.colors.Normalize(vmin=0, vmax=2)

# creating ScalarMappable
sm = plt.cm.ScalarMappable(cmap=cm, norm=norm)
sm.set_array([])

fig, ax = plt.subplots(1,1,figsize=(7,5))
for i in range(mod_vel0.shape[0]-2):
    im = ax.plot(position_per_trial_train[0,i:i+2,0], position_per_trial_train[0,i:i+2,1], color=colors[i+2])
plt.colorbar(sm, ax=ax, label='Velocity')

plt.grid()

plt.xlim(-150,170)
plt.ylim(-120,120)

plt.xlabel('X position')
plt.ylabel('Y position')

#### PCA

In order to perform PCA, we first concatenate the the trials such that the data is of the form (trial x time) x neurons. We then smooth the data with a gaussian kernel.

In [ ]:
# smoothing data with a gaussian kernel
data_stacked = rearrange(spike_data_train, 'trial time neurons -> (trial time) neurons')
data_smooth = scipy.ndimage.gaussian_filter1d(input = data_stacked, sigma=50.0, axis=0)
data_centered = data_smooth - np.mean(data_smooth, axis=0)

print(data_centered.shape)

fig, ax = plt.subplots(1, 1, figsize =(10, 5))
_= ax.plot(data_smooth[:, 0], color='blue')

plt.xlabel('Concatenated trials')

plt.title('Neuron 1')

In [ ]:
# PCA using SVD
u, s, vh = np.linalg.svd(data_centered, full_matrices=False)
u.shape, s.shape

In [ ]:
norm_sv = s**2/np.sum(s**2)
top2sv = np.sum(norm_sv[:2])
print("Total observations explained by the first two principal components: {0:.3f}%".format(top2sv*100))

In [ ]:
plt.plot(20*np.log10(norm_sv), 'o-')
plt.grid()
plt.ylabel("variance explained (dB)"); plt.xlabel("PC (ordered)")

In [ ]:
# visualizing top two PCs

top2u = u[:, :2]
top2s = s[:2]
top2reconstruction = top2u * top2s
print(top2reconstruction.shape)
fig, ax = plt.subplots(1, 1, figsize =(10, 5))
ax.plot(top2reconstruction[:, 0], color='blue')
ax.plot(top2reconstruction[:, 1], color='red')
